In [ ]:
#different ancova codes
#ancova using pinguin
import pandas as pd
df=pd.read_csv("https://reneshbedre.github.io/assets/posts/ancova/ancova_data.csv")
df.head(4)

In [ ]:
%pip install dfply
from dfply import *
# summary statistics for dependent variable yield
df >> group_by(X.genotype) >> summarize(n=X['yield'].count(), mean=X['yield'].mean(), std=X['yield'].std())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, axs = plt.subplots(ncols=3)
sns.scatterplot(data=df, x="height", y="yield", hue=df.genotype.tolist(), ax=axs[0])
sns.boxplot(data=df, x="genotype", y="yield", hue=df.genotype.tolist(), ax=axs[1])
sns.boxplot(data=df, x="genotype", y="height", hue=df.genotype.tolist(), ax=axs[2])
plt.show()


In [ ]:
%pip install pingouin
from pingouin import ancova
ancova(data=df, dv='yield', covar='height', between='genotype')

In [ ]:
import numpy as np
import pandas as pd

# create data
data = pd.DataFrame({'methodology': np.repeat(['A', 'B', 'C'], 4),
                     'current_grade': [67, 88, 75, 85,
                                       92, 77, 74, 88,
                                       91, 88, 82, 80],
                     'test_score': [77, 89, 74, 69,
                                    88, 93, 94, 90,
                                    85, 81, 83, 79]})
# view data
data

In [ ]:
from pingouin import ancova

data = pd.DataFrame({'methodology': np.repeat(['A', 'B', 'C'], 4),
                     'current_grade': [67, 88, 75, 85,
                                       92, 77, 74, 88,
                                       91, 88, 82, 80],
                     'test_score': [77, 89, 74, 69,
                                    88, 93, 94, 90,
                                    85, 81, 83, 79]})

ancova(data=data, dv='test_score', covar='current_grade', between='methodology')

In [ ]:
#ancova using statsmodels
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Define the data for the ANCOVA
df = pd.DataFrame({'dependent_variable' : [8, 7, 9, 11, 10, 12, 14, 13, 15, 16],
   'group' : ["A", "A", "A", "B", "B", "B", "C", "C", "C", "C"],
    'covariate' : [20, 30, 40, 30, 40, 50, 40, 50, 60, 70]})

# Perform the ANCOVA
model = ols('dependent_variable ~ group + covariate', data=df).fit()

# Print the summary of the model
print(model.summary())

In [ ]:
#a more detailed example:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(123)

# Generate age data
age_treatment = np.random.normal(loc=40, scale=10, size=50)
age_control = np.random.normal(loc=40, scale=10, size=50)

# Generate body weight data
baseline_weight_treatment = np.random.normal(loc=80, scale=10, size=50)
baseline_weight_control = np.random.normal(loc=80, scale=10, size=50)
post_weight_treatment = np.random.normal(loc=75, scale=10, size=50)
post_weight_control = np.random.normal(loc=80, scale=10, size=50)

# Combine data into a DataFrame
df = pd.DataFrame({
    'age': np.concatenate((age_treatment, age_control)),
    'baseline_weight': np.concatenate((baseline_weight_treatment, baseline_weight_control)),
    'post_weight': np.concatenate((post_weight_treatment, post_weight_control)),
    'treatment': np.concatenate((np.ones(50), np.zeros(50)))
})

In [ ]:
# Fit ANCOVA model
model = sm.formula.ols('post_weight ~ treatment + age + baseline_weight', data=df).fit()

# Print model summary
print(model.summary())

In [ ]:
# Fit ANCOVA model with interaction term
model_interaction = sm.formula.ols('post_weight ~ treatment * age + baseline_weight', data=df).fit()

# Print model summary
print(model_interaction.summary())

In [ ]:
#ancova in R
install.packages("ggpubr")
library(tidyverse)
library(ggpubr)
library(rstatix)
library(broom)

In [ ]:
# Load and prepare the data
install.packages("datarium")
library(datarium)
data("anxiety", package = "datarium")
anxiety <- anxiety %>%
  select(id, group, t1, t3) %>%
  rename(pretest = t1, posttest = t3)
anxiety[14, "posttest"] <- 19
# Inspect the data by showing one random row by groups
set.seed(123)
anxiety %>% sample_n_by(group, size = 1)

In [ ]:
ggscatter(
  anxiety, x = "pretest", y = "posttest",
  color = "group", add = "reg.line"
  )+
  stat_regline_equation(
    aes(label =  paste(..eq.label.., ..rr.label.., sep = "~~~~"), color = group)
    )

In [ ]:
anxiety %>% anova_test(posttest ~ group*pretest)

In [ ]:
# Fit the model, the covariate goes first
model <- lm(posttest ~ pretest + group, data = anxiety)

In [ ]:
model.metrics <- augment(model)

In [ ]:
head(model.metrics, 3)

In [ ]:
# Assess normality of residuals using shapiro wilk test
shapiro_test(model.metrics$.resid)

In [ ]:
model.metrics %>% levene_test(.resid ~ group)

In [ ]:
res.aov <- anxiety %>% anova_test(posttest ~ pretest + group)
get_anova_table(res.aov)

In [ ]:
install.packages("emmeans")
library(emmeans)
pwc <- anxiety %>%
  emmeans_test(
    posttest ~ group, covariate = pretest,
    p.adjust.method = "bonferroni"
    )
pwc

In [ ]:
# Display the adjusted means of each group
# Also called as the estimated marginal means (emmeans)
get_emmeans(pwc)

In [ ]:
# Visualization: line plots with p-values
pwc <- pwc %>% add_xy_position(x = "group", fun = "mean_se")
ggline(get_emmeans(pwc), x = "group", y = "emmean") +
  geom_errorbar(aes(ymin = conf.low, ymax = conf.high), width = 0.2) +
  stat_pvalue_manual(pwc, hide.ns = TRUE, tip.length = FALSE) +
  labs(
    subtitle = get_test_label(res.aov, detailed = TRUE),
    caption = get_pwc_label(pwc)
  )